<a href="https://colab.research.google.com/github/Turza006/fake-news-detections-ml-approach/blob/master/fack_new_0_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [ ]:
import pandas as pd
from google.colab import drive

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sentence_transformers import SentenceTransformer
import numpy as np




## Importing the dataset

In [ ]:
drive.mount('/content/drive')
real_news = pd.read_csv("/content/drive/MyDrive/news_data/LabeledAuthentic-7K.csv")
fake_news = pd.read_csv("/content/drive/MyDrive/news_data/LabeledFake-1K.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# prompt: print first 5 element from real news

print(fake_news.head())
print(real_news.head())


   articleID                    domain                        date  \
0          1         channeldhaka.news   2019-03-14T13:34:14+00:00   
1          2                 earki.com            সেম্বর ১৭, ২০১৮    
2          3                 earki.com  ২০:৩৯, জানুয়ারি ১৪, ২০১৯    
3          4         channeldhaka.news   2018-06-30T15:56:47+00:00   
4          5  motikontho.wordpress.com   2013-03-05T21:55:45+00:00   

        category    source   relation  \
0  International  Reporter  Unrelated   
1  Miscellaneous  Reporter  Unrelated   
2  Miscellaneous  Reporter  Unrelated   
3         Sports  Reporter  Unrelated   
4  Miscellaneous  Reporter  Unrelated   

                                            headline  \
0                           মুরগির হামলায় শেয়াল নিহত   
1            বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম   
2  বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...   
3                      অবসর নেয়ার ঘোষণা দিলেন মেসি !   
4  মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...

Data Preprocessing (crucial for Bangla text)

In [ ]:
def clean_text(text):
    """Basic text cleaning (customize as needed for Bangla)."""
    # text = re.sub(r'\s+', ' ', text).strip()

    # # Remove punctuation (you can customize this to keep specific characters)
    # text = re.sub(r'[^\u0980-\u09FF\s]', '', text)

    return text

# Clean and label the data

In [ ]:
fake_news['label'] = 1
fake_news['headline'] = fake_news['headline'].apply(clean_text)
fake_news['content'] = fake_news['content'].apply(clean_text)

real_news['label'] = 0
real_news['headline'] = real_news['headline'].apply(clean_text)
real_news['content'] = real_news['content'].apply(clean_text)

# Combine datasets

In [ ]:
combined_data = pd.concat([fake_news, real_news], ignore_index=True)

# Feature Engineering: Combine headline and article, then vectorize.

In [ ]:
combined_data['combined_text'] = combined_data['headline'] + ' ' + combined_data['content']

# TF-IDF Vectorization


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(combined_data['combined_text'])
y = combined_data['label']

# Split data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train and Evaluate Naive Bayes

In [ ]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)
print("Naive Bayes:")
print(classification_report(y_test, nb_predictions))
print("Accuracy:", accuracy_score(y_test, nb_predictions))

Naive Bayes:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1427
           1       0.92      0.31      0.46       274

    accuracy                           0.88      1701
   macro avg       0.90      0.65      0.70      1701
weighted avg       0.89      0.88      0.86      1701

Accuracy: 0.8847736625514403


# Train and Evaluate Random Forest

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42) # Adjust n_estimators as needed
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
print("\nRandom Forest:")
print(classification_report(y_test, rf_predictions))
print("Accuracy:", accuracy_score(y_test, rf_predictions))


Random Forest:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      1427
           1       0.99      0.54      0.70       274

    accuracy                           0.93      1701
   macro avg       0.95      0.77      0.83      1701
weighted avg       0.93      0.93      0.92      1701

Accuracy: 0.9253380364491476


# Example Prediction (Remember to preprocess the input text)


In [ ]:
new_headline = "ড. ইউনূসের এডিটেড ভিডিও দিয়ে জুয়ার অ্যাপের প্রচারণা"
new_article = "সামাজিক যোগাযোগ মাধ্যম ফেসবুকে একটি ভিডিও পোস্ট করা হয়েছে। ভিডিওতে দেখা যাচ্ছে অন্তর্বর্তীকালীন সরকারের প্রধান উপদেষ্টা ও নোবেল বিজয়ী ড. মুহাম্মদ ইউনূস বলেছেন, একটি অ্যাপ তৈরি করা হয়েছে যা বেকারত্ব দূর করতে সাহায্য করবে। একই ভিডিওতে ডাক, টেলিযোগাযোগ ও তথ্যপ্রযুক্তি উপদেষ্টা মো. নাহিদ ইসলামকে বলতে দেখা যায়, সব শিক্ষার্থীরা মুহাম্মদ ইউনূসের অ্যাপটিতে খেলে আয় করছে। ভিডিওটিতে সময় টেলিভিশনের লোগোযুক্ত একজন সংবাদ পাঠিকার এ সংক্রান্ত সংবাদ পাঠের দৃশ্যও যুক্ত করা হয়েছে। ভিডিও সহ এরকম একটি পোস্ট দেখুন এখানে। এছাড়াও আরো একটি ভিডিওতে ড. মুহাম্মদ ইউনূসকে বলতে দেখা যায়, দেশের বর্তমান বিপর্যয়ের কারণে আমি একটি নতুন এপ্লিকেশন তৈরি করার সিদ্ধান্ত নিয়েছি। Crazy Time নামক এই অ্যাপে মাত্র ৫ টাকা বিনিয়োগ করে অনেক ভালো আয় করা যাবে। এমনকি প্রথম দিনেই ৫০ হাজার বা এক লক্ষ টাকা জিততে (আয় করতে) পারেন। ভিডিও সহ এরকম একটি পোস্ট দেখুন এখানে। গত ২০ অক্টোবর 'Yudi liceth Arboleda' নামের পেজ থেকে ভিডিওটি (প্রথমে উল্লিখিত) পোস্ট করা হয়। ভিডিওটি দিয়ে একটি অ্যাপের প্রচারণা চালানো হচ্ছে। পোস্টটির স্ক্রিনশটের (কোলাজ) দেখুন--"

new_text = clean_text(new_headline + ' ' + new_article)
new_text_vec = vectorizer.transform([new_text])

nb_prob = nb_classifier.predict_proba(new_text_vec)[0][1]
rf_prob = rf_classifier.predict_proba(new_text_vec)[0][1]

print(f"\nNaive Bayes probability of being fake: {nb_prob:.2f}")
print(f"Random Forest probability of being fake: {rf_prob:.2f}")


Naive Bayes probability of being fake: 0.10
Random Forest probability of being fake: 0.31
